<a href="https://colab.research.google.com/github/andersonsps-dsa/Energy_Consumption_Customer_Segmentation/blob/main/Energy_Consumption_Customer_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Importações
!pip install -q ucimlrepo
from ucimlrepo import fetch_ucirepo

import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pylab
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cdist, pdist
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
# @title Carregamento do dados:
cons_energ_domicilio_indiv_ = fetch_ucirepo(id=235)
dataset = pd.DataFrame(data=cons_energ_domicilio_indiv_.data.features)

In [ ]:
# @title Tratamento dos dados
if dataset.isnull().values.any() == True:
  print('Possui valores NULOS')
else:
  print('Não possui valores NULOS')

Possui valores NULOS


In [ ]:
dataset.info()
print('\n--------------------------------------------------------------------------------------------------------------------------------------------\n')
dataset.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 9 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   Date                   datetime64[ns]
 1   Time                   object        
 2   Global_active_power    float64       
 3   Global_reactive_power  float64       
 4   Voltage                float64       
 5   Global_intensity       float64       
 6   Sub_metering_1         float64       
 7   Sub_metering_2         float64       
 8   Sub_metering_3         float64       
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 142.5+ MB

--------------------------------------------------------------------------------------------------------------------------------------------



,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-16,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,2006-12-16,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,2006-12-16,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,2006-12-16,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,2006-12-16,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [ ]:
for col in dataset.iloc[:,2:].columns:
  dataset[col] = dataset[col].replace('?', np.nan)
  dataset[col] = dataset[col].astype(float)

dataset['Date'] = pd.to_datetime(dataset['Date'], format='%d/%m/%Y')
dataset['Time'] = pd.to_datetime(dataset['Time'], format='%H:%M:%S').dt.time

dataset.info()
print('---------------------')
dataset.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 9 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   Date                   datetime64[ns]
 1   Time                   object        
 2   Global_active_power    float64       
 3   Global_reactive_power  float64       
 4   Voltage                float64       
 5   Global_intensity       float64       
 6   Sub_metering_1         float64       
 7   Sub_metering_2         float64       
 8   Sub_metering_3         float64       
dtypes: datetime64[ns](1), float64(7), object(1)
memory usage: 142.5+ MB
---------------------


,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,2006-12-16,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,2006-12-16,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2,2006-12-16,17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
3,2006-12-16,17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
4,2006-12-16,17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [ ]:
# @title Analise Estatística Descritiva
atributos = dataset.iloc[:,2:].dropna()

if atributos.isnull().values.any() == True:
  print('Possui valores NULOS')
else:
  print('Não possui valores NULOS')

atributos.describe()

Não possui valores NULOS


,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
count,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06
mean,1.091615e+00,1.237145e-01,2.408399e+02,4.627759e+00,1.121923e+00,1.298520e+00,6.458447e+00
std,1.057294e+00,1.127220e-01,3.239987e+00,4.444396e+00,6.153031e+00,5.822026e+00,8.437154e+00
min,7.600000e-02,0.000000e+00,2.232000e+02,2.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.080000e-01,4.800000e-02,2.389900e+02,1.400000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,6.020000e-01,1.000000e-01,2.410100e+02,2.600000e+00,0.000000e+00,0.000000e+00,1.000000e+00
75%,1.528000e+00,1.940000e-01,2.428900e+02,6.400000e+00,0.000000e+00,1.000000e+00,1.700000e+01
max,1.112200e+01,1.390000e+00,2.541500e+02,4.840000e+01,8.800000e+01,8.000000e+01,3.100000e+01


In [ ]:
correlation_matrix = atributos.corr()
fig = go.Figure(data=go.Heatmap(
                   z=round(correlation_matrix,2).values,
                   x=correlation_matrix.columns,
                   y=correlation_matrix.columns,
                   colorscale='RdBu_r',
                   texttemplate="%{z}",
                   textfont={"size":10}
))
fig.update_layout(title='Heatmap de Correlação')
fig.show()

In [ ]:
amostra1, amostra2 = train_test_split(atributos, train_size= 0.01, random_state=123)
pca = PCA(n_components = 2).fit_transform(amostra1)
pca.shape

In [ ]:
k_range = range(1,15)
k_means_var = [KMeans(n_clusters = k).fit(pca) for k in k_range]
centroids = [x.cluster_centers_ for x in k_means_var]
k_euclid = [cdist(pca, cent, 'euclidean') for cent in centroids]
dist = [np.min(ke, axis = 1) for ke in k_euclid]
wcss = [sum(d**2) for d in dist]
soma_total = sum(pdist(pca)**2)/pca.shape[0]
variacao_explicada = [(i/soma_total)*100 for i in wcss]

In [ ]:
fig_elbow = go.Figure()
fig_elbow.add_trace(go.Scatter(x = list(k_range), y = wcss, name = 'Soma dos quadrados intra cluster'))
fig_elbow.add_trace(go.Scatter(x = list(k_range), y = variacao_explicada, name = 'Variância explicada'))
fig_elbow.update_layout(title = 'Método Elbow', xaxis_title = 'Número de clusters', yaxis_title = 'Valor',
                         width = 800, height = 500)
fig_elbow

In [ ]:
# @title Modelo K-means
n_clusters = 8
modelo1 = KMeans(n_clusters = n_clusters, random_state = 123)
modelo1.fit(pca)

x_min, x_max = pca[:, 0].min() - 5, pca[:, 0].max() + 5
y_min, y_max = pca[:, 1].min() - 5, pca[:, 1].max() + 5

xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.2), np.arange(y_min, y_max, 0.2))
Z = modelo1.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

labels = modelo1.labels_
silhouette = silhouette_score(pca, labels, metric = 'euclidean')
print(f' Silhouette: {silhouette:.6f}')

 Silhouette: 0.808890


In [ ]:
# @title Gráfico dos Clusters e centróides
centroids_np = modelo1.cluster_centers_
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=pca[:, 0], y=pca[:, 1],
    mode='markers',
    marker=dict(size=4, color='darkslateblue'),
    name='Pontos'))

fig.add_trace(go.Scatter(
    x=centroids_np[:, 0], y=centroids_np[:, 1],
    mode='markers+text',
    marker=dict(size=15, symbol='2', color='red', line=dict(width=1)),
    name='Centróides', text=["Centroide"] * len(centroids_np), textposition="top center"))

fig.add_trace(go.Heatmap(
    z=Z,  x=np.linspace(xx.min(), xx.max(), Z.shape[1]),  y=np.linspace(yy.min(), yy.max(), Z.shape[0]),
    colorscale='geyser',
    zsmooth='best',
    colorbar=dict(title='Valores Z')))

inert = modelo1.inertia_
fig.update_layout(
    title=f'Clusters e centróides | Silhouette: {silhouette:.6f} | Centróides: {centroids_np.shape[0]}',
    xaxis_title='Componente Principal 1', yaxis_title='Componente Principal 2',
    xaxis=dict(range=[x_min, x_max], showticklabels=False), yaxis=dict(range=[y_min, y_max], showticklabels=False),
    showlegend=True,
    width=800, height=600)
fig.show()

In [ ]:
# @title Gráfico dos Clusters e centróides
df_cluster = pd.DataFrame(amostra1, columns = dataset.iloc[:,2:].columns)
df_cluster['Cluster'] = modelo1.labels_
df_cluster = df_cluster.reset_index(drop=True)
df_cluster.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3,Cluster
0,1.608,0.300,245.43,6.6,0.0,1.0,0.0,1
1,0.428,0.358,245.31,2.2,1.0,0.0,1.0,1
2,1.650,0.160,239.63,6.8,0.0,0.0,18.0,0
3,0.814,0.000,242.48,3.4,0.0,0.0,0.0,1
4,2.386,0.000,241.03,9.8,0.0,0.0,0.0,1


In [ ]:
Clusters = df_cluster.groupby('Cluster')['Global_active_power'].count()
Clusters

,Global_active_power
Cluster,
0,5956
1,12819
2,326
3,328
4,231
5,242
6,177
7,413
